This section is used to model the Covid-19 disease and plot the results <br>
<b><font size=7>Table of content</font></b><br>
<font size=3.5>
3. [Modeling](#3)<br>
&emsp;3.1. [Methodology](#3.1)<br>
&emsp;3.2. [Modeling databases](#3.2)<br>
&emsp;&emsp;3.2.1. [World database](#3.2.1)<br>
&emsp;&emsp;3.2.2. [VietNam database](#3.2.2)<br>
&emsp;3.3. [Forecasting & Visualization](#3.3)<br>
&emsp;&emsp;3.3.1. [World data](#3.3.1)<br>
&emsp;&emsp;&ensp;a. [By world grand total](#3.3.1.a)<br>
&emsp;&emsp;&ensp;b. [By each country](#3.3.1.b)<br>
&emsp;&emsp;3.3.2. [VietNam data](#3.3.2)<br>
&emsp;&emsp;&ensp;a. [By country grand total](#3.3.2.a)<br>
&emsp;&emsp;&ensp;b. [By each province](#3.3.2.b)<br>
</font>

In [ ]:
import pandas as pd
import numpy as np
import os,time
from IPython.display import clear_output
from functools import partial
from multiprocessing import Manager,Pool
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from covid_science.modeling import SVID_modeling,get_arima_order
from covid_science.workers import wrapper_SVID_predict

<font size=10>**3.Modeling**</font><a name="3"></a><br>
<ins><font size=6>**3.1.Methodology**</font></ins><a name="3.1"></a>

A ordinary differential equation system is built to estimate the spread of Covid-19 virus base on the following information.

The population will be broken down into 4 groups (S+V+I+D=N):
* Susceptible (**S**): The main population which is susceptible to the disease without any protection **(person)**.
* Vaccinated (**V**): The population which has been vaccinated or recovers from the disease and has protection against the disease to some degree **(person)**.
* Infectious (**I**): The infected population, whom can infect the non-infected population **(person)**.
* Dead (**D**): The population which passes away due to the disease (disease-induced mortality) **(person)**.
* Affected population (**N**): The total population which is affected by the disease **(person)**.

<ins>The ODE (ordinary differential equation) model:</ins>
<font size=5>
$$
\begin{align} 
&\frac{dS}{dt} = \frac{-\beta SI}{N} - (\alpha+\mu)S + \alpha_0V +\pi p \\
&\frac{dV}{dt} = \frac{-\beta_vVI}{N} -(\alpha_0 + \mu)V + \theta I + \alpha S + \pi(1-p) \\
&\frac{dI}{dt} = -(\mu +\gamma +\theta)I + \frac{(\beta S + \beta_V V)I}{N} \\
&\frac{dD}{dt} = -\mu D + \gamma I
\end{align} 
$$
</font>

<ins>With initial conditions as follows:</ins>

<font size=5>
$$
\begin{align} 
&S_0 \ge 0 \\
&V_0 \ge 0 \\
&I_0 \ge 0 \\
&D_0 \ge0
\end{align} 
$$
</font>

<ins>And model parameter as follows:</ins>

$\pi$: recruitment rate $(person/time)$ <br>
$p$: recruitment non-vaccinated percent *(%)* <br>
$\alpha$: vaccination rate $(time^{-1})$ <br>
$\alpha_0$: disminissing rate of vaccine effectiveness $(time^{-1})$ <br>
$\beta$: transmission rate $(person/time)$ <br>
$\mu$: natural death rate $(time^{-1})$ <br>
$\gamma$: Covid-19 death rate $(time^{-1})$ <br>
$\theta$: post-infected immunity rate $(time^{-1})$ <br>
$\beta_v = \beta(1-\eta)$: vaccinated tranmission rate $(person/time)$ <br>


By solving the ODE system we can get the stability points:

<font size=4>
$$
\begin{align}
&\frac{dS}{dt} = 0 \\
&\frac{dV}{dt} = 0 \\
&\frac{dI}{dt} = 0 \\
&\frac{dD}{dt} = 0
\end{align}
$$
</font>

[The stability of typical equilibria of smooth ODEs is determined by the sign of real part of eigenvalues of the Jacobian matrix. An equilibrium is asymptotically stable if all eigenvalues have negative real parts; it is unstable if at least one eigenvalue has positive real part.](http://www.scholarpedia.org/article/Equilibrium#:~:text=Jacobian%20Matrix,-The%20stability%20of&text=where%20all%20derivatives%20are%20evaluated,eigenvalue%20has%20positive%20real%20part.)

<font size=5>
$$
J=D_xf = f_x = \frac{\partial f_i}{\partial x_j} =
\begin{pmatrix}
\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} & \ldots & \frac{\partial f_1}{\partial x_n} \\
\frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} & \ldots & \frac{\partial f_2}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial f_n}{\partial x_1} & \frac{\partial f_n}{\partial x_2} & \ldots & \frac{\partial f_n}{\partial x_n} 
\end{pmatrix}
$$
</font>

From there, we can possibly have two answers:

* Disease free equilibrium (DFE): The point at which the population is disease-free,where as:

<font size=4>
$$
\begin{align} 
&S = \frac{\alpha_0\pi + p\mu\pi}{\alpha\mu + \alpha_0\mu +\mu^2} \\
&V = \frac{\alpha\pi + (1-p)\mu\pi}{\alpha\mu + \alpha_0\mu + \mu^2} \\
&I=0 \\
&D=0 \\
&R_0 = \frac{\alpha\beta_v + \alpha_0\beta + p\beta\mu + \beta_v\mu}{(\gamma + \mu + \theta)(\alpha + \alpha_0 + \mu) + p\beta_v\mu} \lt 1
\end{align} 
$$
</font>

* Endemic Equilibrium (EE): The point at which the disease is at equilibrium point (the disease is not totally eradicated and remains in the population).

<font size=4>
$$
\begin{align} 
&S^*\geq 0 \\
&V^*\geq 0 \\
&I^*\geq 0 \\
&D^*\geq 0 \\
&R_0 = \frac{\alpha\beta_v + \alpha_0\beta + p\beta\mu + \beta_v\mu}{(\gamma + \mu + \theta)(\alpha + \alpha_0 + \mu)+p\beta_v\mu} \gt 1
\end{align} 
$$
</font>
*where: $R_0$ is The Basic Reproduction Number that describes the transmissability or contagiousness of an infectious disease. If $R_0$<1, the disease will eventually die out and vice versa.

<ins><font size=6>**3.2.Modeling databases**</font></ins><a name="3.2"></a><br>
<ins><font size=5>**3.2.1.World database**</font></ins><a name="3.2.1"></a><br>
* <ins>Census database</ins>

In [ ]:
#use the mean of birth and death rate from a number of years
mean_years = range(2019,2022)

In [ ]:
birth_df = pd.read_csv('database/Covid-19_raw_data/census_data/crude_birth_rate.csv',encoding='utf-8',
                        usecols=['Region, subregion, country or area *',
                                 'Location code',
                                 'ISO3 Alpha-code',
                                 'Year',
                                 'Crude Birth Rate (births per 1,000 population)'])
birth_df = birth_df.loc[birth_df.Year.isin(mean_years),['Region, subregion, country or area *',
                                                        'Location code',
                                                        'ISO3 Alpha-code',
                                                        'Crude Birth Rate (births per 1,000 population)']].copy()
birth_df = birth_df.groupby(['Region, subregion, country or area *',
                             'Location code',
                             'ISO3 Alpha-code'], dropna=False).mean().reset_index()
birth_df.head(3) # number per 1000 person 

In [ ]:
death_df = pd.read_csv('database/Covid-19_raw_data/census_data/crude_death_rate.csv',encoding='utf-8',
                       usecols=['Region, subregion, country or area *',
                                'Location code',
                                'ISO3 Alpha-code',
                                'Year',
                                'Crude Death Rate (deaths per 1,000 population)'])
death_df = death_df.loc[death_df.Year.isin(mean_years),['Region, subregion, country or area *',
                                                        'Location code',
                                                        'ISO3 Alpha-code',
                                                        'Crude Death Rate (deaths per 1,000 population)']].copy()
death_df = death_df.groupby(['Region, subregion, country or area *',
                             'Location code',
                             'ISO3 Alpha-code'], dropna=False).mean().reset_index()
death_df.head(3) # number per 1000 person 

* <ins>Modeling input database</ins>

In [ ]:
#modeling data of total world
world_df = pd.read_csv("database/Covid-19_modeling_input/WorldData/w_modeling_data.csv",parse_dates=['date'])
# world_df = world_df.iloc[:-1].copy()

#world average birth_rate/1000 person/year
w_b_rate = birth_df.loc[birth_df['Location code']==900,'Crude Birth Rate (births per 1,000 population)'].values[0]
w_birth_rate = (1+(w_b_rate/1000))**(1/365)-1 #(convert to %/day)
#world average death_rate/1000 person/year
w_d_rate = death_df.loc[death_df['Location code']==900,'Crude Death Rate (deaths per 1,000 population)'].values[0]
w_death_rate = (1+(w_d_rate/1000))**(1/365)-1 #(convert to %/day)

#modeling data by each country
bc_df = pd.read_csv("database/Covid-19_modeling_input/WorldData/w_modeling_data_by_country.csv",parse_dates=['date'])

<ins><font size=5>**3.2.2.VietNam database**</font></ins><a name="3.2.2"></a><br>
* <ins>Modeling input database</ins>

In [ ]:
vn_df = pd.read_csv("database/Covid-19_modeling_input/VietNamData/vn_modeling_data.csv",parse_dates=['date'])
# vn_df = vn_df.iloc[:-1].copy()

#average birth_rate/1000 person/year
b_rate = birth_df.loc[birth_df['ISO3 Alpha-code']=='VNM','Crude Birth Rate (births per 1,000 population)'].values[0] 
vn_birth_rate = (1+(b_rate/1000))**(1/365)-1 #(convert to %/day)
#average death_rate/1000 person/year
d_rate = death_df.loc[death_df['ISO3 Alpha-code']=='VNM','Crude Death Rate (deaths per 1,000 population)'].values[0] 
vn_death_rate = (1+(d_rate/1000))**(1/365)-1 #(convert to %/day)

#modeling data by each province
vn_bp_df = pd.read_csv("database/Covid-19_modeling_input/VietNamData/vn_modeling_data_by_province.csv",parse_dates=['date'])

<ins><font size=6>**3.3.Forecasting & Visualization**</font></ins><a name="3.3"></a><br>
<ins><font size=5>**3.3.1.World data**</font></ins><a name="3.3.1"></a><br>
<font size=4><ins>**a. By world grand total**</ins></font><a name="3.3.1.a"></a><br>
* <ins>General input parameters</ins>

In [ ]:
#input for world data processing
w_p = 1.0

#input for SVID_modeling
w_start_date = world_df.iloc[-1].date
w_days = 7
w_model_number=[1,2,3]
w_scoring = 'neg_mean_absolute_percentage_error'
w_greater_better = False
w_best_model = True #if no best model is chosen, the data of the first model will be chosen for plotting
w_multioutput = "variance_weighted"
w_preset_order = None #for application, this can be preset for faster model calculation
w_param_kwargs = None
w_plot_point = 30

#conversion to real data
w_r_protect_time = 150
w_n_days = 1000 

* <ins>Data processing and forecast SVID model future values</ins>

In [ ]:
start = time.time()
w_manager = Manager()
w_counter= w_manager.Array('i',[0,0])
w_time_list = w_manager.list([])
w_error_dict = w_manager.dict()
w_shared_output = w_manager.Namespace()
w_shared_output.shared_df = pd.DataFrame()
w_shared_output.equil_df = pd.DataFrame()
w_lock = w_manager.Lock()

#extra argument for function input
w_input_kwargs = {'predict_days':w_days,
                   'date_col':'date',
                   'initial_state':['S0','V0','I0','D0'],
                   'model_param':['beta','beta_v','gamma','theta','alpha','alpha0','pi'],
                   'model_number':w_model_number,
                   'best_model':w_best_model,
                   'scoring':w_scoring,
                   'multioutput':w_multioutput,
                   'greater_better':w_greater_better,
                   'preset_arima_order': w_preset_order, #for fast modeling, if set to None, it will be much slower 
                   'param_kwargs':w_param_kwargs
                  }

#worker function
worker_func = partial(wrapper_SVID_predict,
                      shared_count=w_counter,
                      time_list = w_time_list,
                      error_dict=w_error_dict,
                      shared_output=w_shared_output,
                      lock=w_lock,
                      input_data=(world_df,'iso_code'),
                      start_date=w_start_date,
                      birth_data=w_birth_rate,
                      death_data=w_death_rate,
                      p=w_p,
                      r_protect_time=w_r_protect_time,
                      n_days = w_n_days,
                      **w_input_kwargs)

worker_func('OWID_WRL')

w_predict_df = w_shared_output.shared_df.copy()
w_predict_df = w_predict_df.sort_values(['iso_code','method','date']).reset_index(drop=True)
w_equil_df = w_shared_output.equil_df.copy()
w_error_result = dict(w_error_dict)
w_time = list(w_time_list)

w_manager.shutdown()

end = time.time()

print(f"Total processing time: {end-start}s")

* <ins>Store modeling data</ins>

In [ ]:
folder_path = "database/Covid-19_modeling_output/WorldData"
if os.path.exists(str(folder_path)):
    pass
else:
    os.makedirs(str(folder_path))
    print("New folder created.")
w_predict_df.to_csv(folder_path +"/"+"w_prediction.csv",encoding="utf-8",index=False)
w_equil_df.to_csv(folder_path +"/"+"w_equilibrium.csv",encoding="utf-8",index=False)
w_predict_df.head(3)

* <ins>Plot forecast data</ins>

In [ ]:
##making frame for the subplots

w_specs = [[{},{}],
            [{},{}],
            [{},{}],
            [{},{}],
            [{},{}],
            [{"colspan": 2},None]]
sub_title = ["Forecasted S",
             "Forecasted V",
             "Forecasted I",
             "Forecasted D",
             "Forecasted daily_case_non_vaccinated",
             "Forecasted daily_case_vaccinated",
             "Forecasted death_avg",
             "Forecasted recovery_case",
             "Forecasted new_full_vaccinated",
             "Forecasted new_boost_req",
             "Equilibrium plot"]
w_fig = make_subplots(rows=6,cols=2,specs=w_specs,subplot_titles=sub_title,vertical_spacing=0.05,horizontal_spacing=0.05)

for plot_color in ['blue','red']:
    for i in range(0,10):
        if i%2==0:
            add_col=1
            showlegend=False
        else:
            add_col=2
            showlegend=True
        add_row = (i//2)+1
        if plot_color=='blue':
            data_name="real"
        else:
            data_name="forecast"
        w_fig.add_trace(go.Scatter(name=data_name,line={'color':plot_color},legendgroup=str(i),showlegend=showlegend,mode='lines'),
                         row=add_row,col=add_col)
w_fig.add_trace(go.Scatter(name="S",line={'color':'green'},legendgroup='10',mode='lines'),row=6,col=1)
w_fig.add_trace(go.Scatter(name="V",line={'color':'blue'},legendgroup='10',mode='lines'),row=6,col=1)
w_fig.add_trace(go.Scatter(name="I",line={'color':'orange'},legendgroup='10',mode='lines'),row=6,col=1)

##create buttons data for dropdown list

w_real_data = (world_df.loc[world_df.date<=w_start_date].iloc[-w_plot_point:,[0,4,5,6,7,19,20,15,16,17,18]].to_numpy())

#forecast data
if w_best_model:
    w_forecast_data = w_predict_df.loc[w_predict_df.iso_code=='OWID_WRL'].\
    loc[:,['S0','V0','I0','D0','daily_case_non_vaccinated','daily_case_vaccinated','death_avg','recovery_case','new_full_vaccinated','new_boost_req']].to_numpy()
else: #if no best model is chosen, the data of the first model will be chosen
    w_forecast_data = w_predict_df.loc[(w_predict_df.iso_code=='OWID_WRL')&(w_predict_df.method==w_model_number[0])].\
    loc[:,['S0','V0','I0','D0','daily_case_non_vaccinated','daily_case_vaccinated','death_avg','recovery_case','new_full_vaccinated','new_boost_req']].to_numpy()
w_forecast_date = pd.date_range(start=w_start_date,periods=w_days+1)

#get reproduction rate
w_R0 = world_df.loc[world_df.date==w_start_date,['daily_R0', 'avg_R0']].to_numpy().squeeze().tolist()

w_date_range = pd.date_range(w_start_date,periods=w_n_days)

x_list=[]
x_list.extend([w_real_data[:,0]]*10)
x_list.extend([w_forecast_date]*10)
x_list.extend([w_equil_df.date.to_numpy()]*3)

y_list = []
y_list.extend(w_real_data[:,1:].T)
y_list.extend(w_forecast_data.T)
y_list.extend(w_equil_df[['S','V','I']].to_numpy().T)

#update plot data
for i,data in enumerate(zip(x_list,y_list)):
    w_fig.data[i].update(x=data[0],y=data[1])

w_fig.layout.annotations[10].update(text=f'Equilibrium plot (daily_R0:{round(w_R0[0],2)} - average_R0:{round(w_R0[1],2)})')
clear_output()
w_fig.update_layout(title_text="Total world forecast",title_x=0.5,title_font_size=50,
                   height=2000,
                   legend_tracegroupgap=285)

<font size=4><ins>**b. By each country**</ins></font><a name="3.3.1.b"></a><br>
* <ins>General input parameters</ins>

In [ ]:
#input for each country data processing
bc_p = 1.0

#input for SVID_modeling
bc_start_date = None 
bc_days = 7
bc_model_number=[1,2,3]
bc_scoring = 'neg_mean_absolute_percentage_error'
bc_greater_better = False
bc_best_model = True
bc_multioutput = "variance_weighted"
bc_param_kwargs = None
bc_plot_point = 30

#conversion to real data
bc_r_protect_time = 150
bc_n_days = 1000 

# get general ARIMA order, seasonal order from world total data using for each country
bc_preset_order = get_arima_order(input_df=world_df,
                                   cols=['beta','beta_v','gamma','theta','alpha','alpha0','pi'],
                                   start=world_df.loc[world_df.date==w_start_date].index[0])

bc_max_worker = 6

* <ins>Data processing and forecast SVID model future values</ins>

In [ ]:
start_cell = time.perf_counter()

bc_manager = Manager()
bc_counter= bc_manager.Array('i',[0,0])
bc_time_list = bc_manager.list([])
bc_error_dict = bc_manager.dict()
bc_shared_output = bc_manager.Namespace()
bc_shared_output.shared_df = pd.DataFrame()
bc_shared_output.equil_df = pd.DataFrame()
bc_lock = bc_manager.Lock()

#list of country iso_code
bc_iso_code = bc_df.iso_code.unique()


#extra argument for function input
bc_input_kwargs = {'predict_days':bc_days,
                   'date_col':'date',
                   'initial_state':['S0','V0','I0','D0'],
                   'model_param':['beta','beta_v','gamma','theta','alpha','alpha0','pi'],
                   'model_number':bc_model_number,
                   'best_model':bc_best_model,
                   'scoring':bc_scoring,
                   'multioutput':bc_multioutput,
                   'greater_better':bc_greater_better,
                   'preset_arima_order': bc_preset_order, #for fast modeling, if set to None, it will be much slower 
                   'param_kwargs':bc_param_kwargs
                  }

#worker function
worker_func = partial(wrapper_SVID_predict,
                      shared_count=bc_counter,
                      time_list = bc_time_list,
                      error_dict=bc_error_dict,
                      shared_output=bc_shared_output,
                      lock=bc_lock,
                      input_data=(bc_df,'iso_code'),
                      start_date=bc_start_date,
                      birth_data=(birth_df,'ISO3 Alpha-code','Crude Birth Rate (births per 1,000 population)'),
                      death_data=(death_df,'ISO3 Alpha-code','Crude Death Rate (deaths per 1,000 population)'),
                      p=bc_p,
                      r_protect_time=bc_r_protect_time,
                      n_days = bc_n_days,
                      **bc_input_kwargs)

executor= Pool(processes=bc_max_worker)
start_worker = time.perf_counter()
pool_result = executor.map_async(worker_func,bc_iso_code)

max_country = len(bc_iso_code)
while bc_counter[0]<max_country:
    time.sleep(1)
    n_done = bc_counter[0]
    n_wait = max_country
    total_time = round(sum(bc_time_list),2)
    if n_done!=0:
        avg_time = round(total_time/n_done,2)
    if n_done!=0:
        remain_country = n_wait - n_done
        remain_time = round(remain_country*avg_time,2)
        real_remain = round(remain_time/bc_max_worker,2)
    else:
        remain_country = "estimating..."
        remain_time = "-- "
        real_remain = "-- "
        
    elapsed_time = round(time.perf_counter()-start_worker,2)
    a_string = (f"Elapsed time: {elapsed_time}s    " +
                f"Total workers processing time: {total_time}s    " + 
                f" Remaining countries: {remain_country}    " + 
                f" Estimated remaining worker time: {remain_time}s    " +
                f" Estimated remaining real time: {real_remain}s")
    
    print(" "*(len(a_string)+10),end="\r")
    print(a_string,end="\r")

print(" "*(len(a_string)+10),end="\r")
print("---Forecast processing completed---")
print(f"Number of workers with error happened: {bc_counter[1]}")
print(f"Total worker processing time taken: {round(sum(bc_time_list),2)}s")

bc_predict_df = bc_shared_output.shared_df.copy()
bc_predict_df = bc_predict_df.sort_values(['iso_code','method','date']).reset_index(drop=True)
bc_equil_df = bc_shared_output.equil_df.copy()
bc_equil_df = bc_equil_df.sort_values(['iso_code','date']).reset_index(drop=True)
bc_error_result = dict(bc_error_dict)
bc_time = list(bc_time_list)

executor.close()
executor.join()
bc_manager.shutdown()

end_cell = time.perf_counter()
print(f"Total elapsed time on this code block: {round(end_cell-start_cell,2)}s")


* <ins>Store modeling data</ins>

In [ ]:
folder_path = "database/Covid-19_modeling_output/WorldData"
if os.path.exists(str(folder_path)):
    pass
else:
    os.makedirs(str(folder_path))
    print("New folder created.")
bc_predict_df.to_csv(folder_path +"/"+"w_prediction_by_country.csv",encoding="utf-8",index=False)
bc_equil_df.to_csv(folder_path +"/"+"w_equilibrium_by_country.csv",encoding="utf-8",index=False)
bc_predict_df.head(3)

* <ins>Plot forecast data</ins>

In [ ]:
##making frame for the subplots
bc_col_loc1 = [bc_df.columns.get_loc('date'),
              bc_df.columns.get_loc('S0'),
              bc_df.columns.get_loc('V0'),
              bc_df.columns.get_loc('I0'),
              bc_df.columns.get_loc('D0'),
              bc_df.columns.get_loc('daily_case_non_vaccinated'),
              bc_df.columns.get_loc('daily_case_vaccinated'),
              bc_df.columns.get_loc('death_avg'),
              bc_df.columns.get_loc('recovery_case'),
              bc_df.columns.get_loc('new_full_vaccinated'),
              bc_df.columns.get_loc('new_boost_req')]
bc_col_loc2 = [bc_predict_df.columns.get_loc('date'),
              bc_predict_df.columns.get_loc('S0'),
              bc_predict_df.columns.get_loc('V0'),
              bc_predict_df.columns.get_loc('I0'),
              bc_predict_df.columns.get_loc('D0'),
              bc_predict_df.columns.get_loc('daily_case_non_vaccinated'),
              bc_predict_df.columns.get_loc('daily_case_vaccinated'),
              bc_predict_df.columns.get_loc('death_avg'),
              bc_predict_df.columns.get_loc('recovery_case'),
              bc_predict_df.columns.get_loc('new_full_vaccinated'),
              bc_predict_df.columns.get_loc('new_boost_req')]

bc_specs = [[{},{}],
            [{},{}],
            [{},{}],
            [{},{}],
            [{},{}],
            [{"colspan": 2},None]]
sub_title = ["Forecasted S",
             "Forecasted V",
             "Forecasted I",
             "Forecasted D",
             "Forecasted daily_case_non_vaccinated",
             "Forecasted daily_case_vaccinated",
             "Forecasted death_avg",
             "Forecasted recovery_case",
             "Forecasted new_full_vaccinated",
             "Forecasted new_boost_req",
             "Equilibrium plot"]
bc_fig = make_subplots(rows=6,cols=2,specs=bc_specs,subplot_titles=sub_title,vertical_spacing=0.05,horizontal_spacing=0.05)
bc_fig['layout']['annotations'] += (dict(text="Country:", x=-0.02, xref="paper",xanchor='left', y=1.042, yref="paper",yanchor="top",
                                                                                     align="left", showarrow=False),)

for plot_color in ['blue','red']:
    for i in range(0,10):
        if i%2==0:
            add_col=1
            showlegend=False
        else:
            add_col=2
            showlegend=True
        add_row = (i//2)+1
        if plot_color=='blue':
            data_name="real"
        else:
            data_name="forecast"
        bc_fig.add_trace(go.Scatter(name=data_name,line={'color':plot_color},legendgroup=str(i),showlegend=showlegend,mode='lines'),
                         row=add_row,col=add_col)
bc_fig.add_trace(go.Scatter(name="S",line={'color':'green'},legendgroup='10',mode='lines'),row=6,col=1)
bc_fig.add_trace(go.Scatter(name="V",line={'color':'blue'},legendgroup='10',mode='lines'),row=6,col=1)
bc_fig.add_trace(go.Scatter(name="I",line={'color':'orange'},legendgroup='10',mode='lines'),row=6,col=1)

##create buttons data for dropdown list

update_menu=[]
country_list = bc_df[['iso_code','location']].drop_duplicates(ignore_index=True)
annotation_list = []
for note in bc_fig.layout.annotations:
    annotation_list.append(go.layout.Annotation(note))
    
for code in bc_predict_df.iso_code.unique():
    if bc_start_date is not None:
        bc_start_stamp = pd.Timestamp(bc_start_date)
        real_data = (bc_df.loc[bc_df.iso_code==code]
                     .loc[(bc_df.date<=bc_start_stamp)&(bc_df.date>=(bc_start_stamp-pd.Timedelta(bc_plot_point,unit="D")))]
                     .iloc[-bc_plot_point:,bc_col_loc1].to_numpy())
    else:
        bc_start_stamp = pd.Timestamp(bc_df.loc[bc_df.iso_code==code].date.values[-1])
        real_data = bc_df.loc[bc_df.iso_code==code].iloc[-bc_plot_point:,bc_col_loc1].to_numpy()
    
    #forecast data
    if bc_best_model:
        forecast_data = bc_predict_df.loc[bc_predict_df.iso_code==code].iloc[:,bc_col_loc2].to_numpy()
    else: #if no best model is chosen, the data of the first model will be chosen
        forecast_data = bc_predict_df.loc[(bc_predict_df.iso_code==code)&(bc_predict_df.method==bc_model_number[0])].iloc[:,bc_col_loc2].to_numpy()
    
    #get reproduction rate
    bc_R0 = bc_df.loc[bc_df.iso_code==code].loc[bc_df.date==bc_start_stamp,['daily_R0', 'avg_R0']].to_numpy().squeeze().tolist()
        
    x_list=[]
    x_list.extend([real_data[:,0]]*10)
    x_list.extend([forecast_data[:,0]]*10)
    x_list.extend([bc_equil_df.loc[bc_equil_df.iso_code==code,'date'].to_numpy()]*3)
    
    y_list = []
    y_list.extend(real_data[:,1:].T)
    y_list.extend(forecast_data[:,1:].T)
    y_list.extend(bc_equil_df.loc[bc_equil_df.iso_code==code,['S','V','I']].to_numpy().T)
    
    #The main title of each button
    plot_title=country_list.loc[country_list.iso_code==code,'location'].values[0]
    
    button_annotation = annotation_list.copy()
    button_annotation[10] = go.layout.Annotation(bc_fig.layout.annotations[10]).update(text=f'Equilibrium plot (daily_R0:{round(bc_R0[0],2)} - average_R0:{round(bc_R0[1],2)})')
    button =  dict(label=code,
                   method="update",
                   args=[{"y":y_list,
                          "x":x_list},
                         {"title":{"text":plot_title,"x":0.5,"font":{"size":50}},
                          "annotations":button_annotation}
                   ]
                  )
    update_menu.append(button)
    
##plotting

bc_fig.update_layout(updatemenus=[dict(active=-1,
                                      buttons=update_menu,
                                      pad={"r": 10, "t": 10},
                                      showactive=True,
                                      xanchor="left",
                                      x=0.05,
                                      yanchor="top",
                                      y=1.05)
                                  ],
                     legend_tracegroupgap=285,
                     height=2000
                    )
clear_output()
bc_fig.show()

<ins><font size=5>**3.3.2.VietNam data**</font></ins><a name="3.3.2"></a><br>
<font size=4><ins>**a. By country grand total**</ins></font><a name="3.3.2.a"></a><br>
* <ins>General input parameters</ins>

In [ ]:
#input for Vietnam data processing
vn_p=1.0

In [ ]:
#input for SVID_modeling
vn_start_date = vn_df.iloc[-1].date
vn_days = 7
vn_model_number=[1,2,3]
vn_scoring = 'neg_mean_absolute_percentage_error'
vn_greater_better = False
vn_best_model = True #if no best model is chosen, the data of the first model will be chosen for plotting
vn_multioutput = "variance_weighted"
vn_preset_order = None #for application, this can be preset for faster model calculation
vn_param_kwargs = None
vn_plot_point = 30

#conversion to real data
vn_r_protect_time = 150
vn_n_days = 1000

* <ins>Data processing and forecast SVID model future values</ins>

In [ ]:
start = time.time()

vn_manager = Manager()
vn_counter= vn_manager.Array('i',[0,0])
vn_time_list = vn_manager.list([])
vn_error_dict = vn_manager.dict()
vn_shared_output = vn_manager.Namespace()
vn_shared_output.shared_df = pd.DataFrame()
vn_shared_output.equil_df = pd.DataFrame()

vn_lock = vn_manager.Lock()


#extra argument for function input
vn_input_kwargs = {'predict_days':vn_days,
                   'date_col':'date',
                   'initial_state':['S0','V0','I0','D0'],
                   'model_param':['beta','beta_v','gamma','theta','alpha','alpha0','pi'],
                   'model_number':vn_model_number,
                   'best_model':vn_best_model,
                   'scoring':vn_scoring,
                   'multioutput':vn_multioutput,
                   'greater_better':vn_greater_better,
                   'preset_arima_order': vn_preset_order, #for fast modeling, if set to None, it will be much slower 
                   'param_kwargs':vn_param_kwargs
                  }

#worker function
worker_func = partial(wrapper_SVID_predict,
                      shared_count=vn_counter,
                      time_list = vn_time_list,
                      error_dict=vn_error_dict,
                      shared_output=vn_shared_output,
                      lock=vn_lock,
                      input_data=(vn_df,'iso_code'),
                      start_date=vn_start_date,
                      birth_data=vn_birth_rate,
                      death_data=vn_death_rate,
                      p=vn_p,
                      r_protect_time=vn_r_protect_time,
                      n_days = vn_n_days,
                      **vn_input_kwargs)

worker_func('VNM')

vn_predict_df = vn_shared_output.shared_df.copy()
vn_predict_df = vn_predict_df.sort_values(['iso_code','method','date']).reset_index(drop=True)
vn_equil_df = vn_shared_output.equil_df.copy()
vn_error_result = dict(vn_error_dict)
vn_time = list(vn_time_list)

vn_manager.shutdown()

end = time.time()

print(f"Total processing time: {end-start}s")

* <ins>Store modeling data</ins>

In [ ]:
folder_path = "database/Covid-19_modeling_output/VietNamData"
if os.path.exists(str(folder_path)):
    pass
else:
    os.makedirs(str(folder_path))
    print("New folder created.")
vn_predict_df.to_csv(folder_path +"/"+"vn_prediction.csv",encoding="utf-8",index=False)
vn_equil_df.to_csv(folder_path +"/"+"vn_equilibrium.csv",encoding="utf-8",index=False)
vn_predict_df.head(3)

* <ins>Plot forecast data</ins>

In [ ]:
##making frame for the subplots

vn_specs = [[{},{}],
            [{},{}],
            [{},{}],
            [{},{}],
            [{},{}],
            [{"colspan": 2},None]]
sub_title = ["Forecasted S",
             "Forecasted V",
             "Forecasted I",
             "Forecasted D",
             "Forecasted daily_case_non_vaccinated",
             "Forecasted daily_case_vaccinated",
             "Forecasted death_avg",
             "Forecasted recovery_case",
             "Forecasted new_full_vaccinated",
             "Forecasted new_boost_req",
             "Equilibrium plot"]
vn_fig = make_subplots(rows=6,cols=2,specs=vn_specs,subplot_titles=sub_title,vertical_spacing=0.05,horizontal_spacing=0.05)

for plot_color in ['blue','red']:
    for i in range(0,10):
        if i%2==0:
            add_col=1
            showlegend=False
        else:
            add_col=2
            showlegend=True
        add_row = (i//2)+1
        if plot_color=='blue':
            data_name="real"
        else:
            data_name="forecast"
        vn_fig.add_trace(go.Scatter(name=data_name,line={'color':plot_color},legendgroup=str(i),showlegend=showlegend,mode='lines'),
                         row=add_row,col=add_col)
vn_fig.add_trace(go.Scatter(name="S",line={'color':'green'},legendgroup='10',mode='lines'),row=6,col=1)
vn_fig.add_trace(go.Scatter(name="V",line={'color':'blue'},legendgroup='10',mode='lines'),row=6,col=1)
vn_fig.add_trace(go.Scatter(name="I",line={'color':'orange'},legendgroup='10',mode='lines'),row=6,col=1)

##create buttons data for dropdown list

vn_real_data = (vn_df.loc[vn_df.date<=vn_start_date].iloc[-vn_plot_point:,[0,2,3,4,5,17,18,13,14,15,16]].to_numpy())

#forecast data
if vn_best_model:
    vn_forecast_data = vn_predict_df.loc[vn_predict_df.iso_code=='VNM'].\
    loc[:,['S0','V0','I0','D0','daily_case_non_vaccinated','daily_case_vaccinated','death_avg','recovery_case','new_full_vaccinated','new_boost_req']].to_numpy()
else: #if no best model is chosen, the data of the first model will be chosen
    vn_forecast_data = vn_predict_df.loc[(vn_predict_df.iso_code=='VNM')&(vn_predict_df.method==vn_model_number[0])].\
    loc[:,['S0','V0','I0','D0','daily_case_non_vaccinated','daily_case_vaccinated','death_avg','recovery_case','new_full_vaccinated','new_boost_req']].to_numpy()
vn_forecast_date = pd.date_range(start=vn_start_date,periods=vn_days+1)

#get reproduction rate
vn_R0 = vn_df.loc[vn_df.date==vn_start_date,['daily_R0', 'avg_R0']].to_numpy().squeeze().tolist()

x_list=[]
x_list.extend([vn_real_data[:,0]]*10)
x_list.extend([vn_forecast_date]*10)
x_list.extend([vn_equil_df.date.to_numpy()]*3)

y_list = []
y_list.extend(vn_real_data[:,1:].T)
y_list.extend(vn_forecast_data.T)
y_list.extend(vn_equil_df[['S','V','I']].to_numpy().T)

#update plot data
for i,data in enumerate(zip(x_list,y_list)):
    vn_fig.data[i].update(x=data[0],y=data[1])

vn_fig.layout.annotations[10].update(text=f'Equilibrium plot (daily_R0:{round(vn_R0[0],2)} - average_R0:{round(vn_R0[1],2)})')
clear_output()
vn_fig.update_layout(title_text="VietNam forecast",title_x=0.5,title_font_size=50,
                   height=2000,
                   legend_tracegroupgap=285)

<font size=4><ins>**b. By each province**</ins></font><a name="3.3.2.b"></a><br>
* <ins>General input parameters</ins>

In [ ]:
#input for each province data processing
vn_bp_p = 1.0

#input for SVID_modeling
vn_bp_start_date = None
vn_bp_days = 7
vn_bp_model_number=[1,2,3]
vn_bp_scoring = 'neg_mean_absolute_percentage_error'
vn_bp_greater_better = False
vn_bp_best_model = True #if no best model is chosen, the data of the first model will be chosen for plotting
vn_bp_multioutput = "variance_weighted"
vn_bp_param_kwargs = None
vn_bp_plot_point = 30

#conversion to real data
vn_bp_r_protect_time = 150
vn_bp_n_days = 1000 

# get general ARIMA order, seasonal order from world total data using for each country
vn_bp_preset_order = get_arima_order(input_df=vn_df,
                                   cols=['beta','beta_v','gamma','theta','alpha','alpha0','pi'],
                                   start=vn_df.loc[vn_df.date==vn_start_date].index[0])

vn_bp_max_worker = 6

* <ins>Data processing and forecast SVID model future values</ins>

In [ ]:
start_cell = time.perf_counter()

vn_bp_manager = Manager()
vn_bp_counter= vn_bp_manager.Array('i',[0,0])
vn_bp_time_list = vn_bp_manager.list([])
vn_bp_error_dict = vn_bp_manager.dict()
vn_bp_shared_output = vn_bp_manager.Namespace()
vn_bp_shared_output.shared_df = pd.DataFrame()
vn_bp_shared_output.equil_df = pd.DataFrame()
vn_bp_lock = vn_bp_manager.Lock()

#list of province code
vn_bp_code = vn_bp_df.id.unique()


#extra argument for function input
vn_bp_input_kwargs = {'predict_days':vn_bp_days,
                   'date_col':'date',
                   'initial_state':['S0','V0','I0','D0'],
                   'model_param':['beta','beta_v','gamma','theta','alpha','alpha0','pi'],
                   'model_number':vn_bp_model_number,
                   'best_model':vn_bp_best_model,
                   'scoring':vn_bp_scoring,
                   'multioutput':vn_bp_multioutput,
                   'greater_better':vn_bp_greater_better,
                   'preset_arima_order': vn_bp_preset_order, #for fast modeling, if set to None, it will be much slower 
                   'param_kwargs':vn_bp_param_kwargs
                  }

#worker function
worker_func = partial(wrapper_SVID_predict,
                      shared_count=vn_bp_counter,
                      time_list = vn_bp_time_list,
                      error_dict=vn_bp_error_dict,
                      shared_output=vn_bp_shared_output,
                      lock=vn_bp_lock,
                      input_data=(vn_bp_df,'id'),
                      start_date=vn_bp_start_date,
                      birth_data=vn_birth_rate,
                      death_data=vn_death_rate,
                      p=vn_bp_p,
                      r_protect_time=vn_bp_r_protect_time,
                      n_days = vn_bp_n_days,
                      **vn_bp_input_kwargs)

executor= Pool(processes=vn_bp_max_worker)
start_worker = time.perf_counter()
pool_result = executor.map_async(worker_func,vn_bp_code)

max_province = len(vn_bp_code)
while vn_bp_counter[0]<max_province:
    time.sleep(1)
    n_done = vn_bp_counter[0]
    n_wait = max_province
    total_time = round(sum(vn_bp_time_list),2)
    if n_done!=0:
        avg_time = round(total_time/n_done,2)
    if n_done!=0:
        remain_province = n_wait - n_done
        remain_time = round(remain_province*avg_time,2)
        real_remain = round(remain_time/vn_bp_max_worker,2)
    else:
        remain_province = "estimating..."
        remain_time = "-- "
        real_remain = "-- "
        
    elapsed_time = round(time.perf_counter()-start_worker,2)
    a_string = (f"Elapsed time: {elapsed_time}s    " +
                f"Total workers processing time: {total_time}s    " + 
                f" Remaining provinces: {remain_province}    " + 
                f" Estimated remaining worker time: {remain_time}s    " +
                f" Estimated remaining real time: {real_remain}s")
    
    print(" "*(len(a_string)+10),end="\r")
    print(a_string,end="\r")

print(" "*(len(a_string)+10),end="\r")
print("---Forecast processing completed---")
print(f"Number of workers with error happened: {vn_bp_counter[1]}")
print(f"Total worker processing time taken: {round(sum(vn_bp_time_list),2)}s")

vn_bp_predict_df = vn_bp_shared_output.shared_df.copy()
vn_bp_predict_df = vn_bp_predict_df.sort_values(['id','method','date']).reset_index(drop=True)
vn_bp_equil_df = vn_bp_shared_output.equil_df.copy()
vn_bp_equil_df = vn_bp_equil_df.sort_values(['id','date']).reset_index(drop=True)
vn_bp_error_result = dict(vn_bp_error_dict)
vn_bp_time = list(vn_bp_time_list)

executor.close()
executor.join()
vn_bp_manager.shutdown()

end_cell = time.perf_counter()
print(f"Total elapsed time on this code block: {round(end_cell-start_cell,2)}s")


* <ins>Store modeling data</ins>

In [ ]:
folder_path = "database/Covid-19_modeling_output/VietNamData"
if os.path.exists(str(folder_path)):
    pass
else:
    os.makedirs(str(folder_path))
    print("New folder created.")
vn_bp_predict_df.to_csv(folder_path +"/"+"vn_prediction_by_province.csv",encoding="utf-8",index=False)
vn_bp_equil_df.to_csv(folder_path +"/"+"vn_equilibrium_by_province.csv",encoding="utf-8",index=False)
vn_bp_predict_df.head(3)

* <ins>Plot forecast data</ins>

In [ ]:
##making frame for the subplots

vn_col_loc1 = [vn_bp_df.columns.get_loc('date'),
              vn_bp_df.columns.get_loc('S0'),
              vn_bp_df.columns.get_loc('V0'),
              vn_bp_df.columns.get_loc('I0'),
              vn_bp_df.columns.get_loc('D0'),
              vn_bp_df.columns.get_loc('daily_case_non_vaccinated'),
              vn_bp_df.columns.get_loc('daily_case_vaccinated'),
              vn_bp_df.columns.get_loc('death_avg'),
              vn_bp_df.columns.get_loc('recovery_case'),
              vn_bp_df.columns.get_loc('new_full_vaccinated'),
              vn_bp_df.columns.get_loc('new_boost_req')]
vn_col_loc2 = [vn_bp_predict_df.columns.get_loc('date'),
              vn_bp_predict_df.columns.get_loc('S0'),
              vn_bp_predict_df.columns.get_loc('V0'),
              vn_bp_predict_df.columns.get_loc('I0'),
              vn_bp_predict_df.columns.get_loc('D0'),
              vn_bp_predict_df.columns.get_loc('daily_case_non_vaccinated'),
              vn_bp_predict_df.columns.get_loc('daily_case_vaccinated'),
              vn_bp_predict_df.columns.get_loc('death_avg'),
              vn_bp_predict_df.columns.get_loc('recovery_case'),
              vn_bp_predict_df.columns.get_loc('new_full_vaccinated'),
              vn_bp_predict_df.columns.get_loc('new_boost_req')]

vn_bp_specs = [[{},{}],
              [{},{}],
              [{},{}],
              [{},{}],
              [{},{}],
              [{"colspan": 2},None]]
sub_title = ["Forecasted S",
             "Forecasted V",
             "Forecasted I",
             "Forecasted D",
             "Forecasted daily_case_non_vaccinated",
             "Forecasted daily_case_vaccinated",
             "Forecasted death_avg",
             "Forecasted recovery_case",
             "Forecasted new_full_vaccinated",
             "Forecasted new_boost_req",
             "Equilibrium plot"]
vn_bp_fig = make_subplots(rows=6,cols=2,specs=vn_bp_specs,subplot_titles=sub_title,vertical_spacing=0.05,horizontal_spacing=0.05)
vn_bp_fig['layout']['annotations'] += (dict(text="Province:", x=-0.02, xref="paper",xanchor='left', y=1.042, yref="paper",yanchor="top",
                                                                                     align="left", showarrow=False),)

for plot_color in ['blue','red']:
    for i in range(0,10):
        if i%2==0:
            add_col=1
            showlegend=False
        else:
            add_col=2
            showlegend=True
        add_row = (i//2)+1
        if plot_color=='blue':
            data_name="real"
        else:
            data_name="forecast"
        vn_bp_fig.add_trace(go.Scatter(name=data_name,line={'color':plot_color},legendgroup=str(i),showlegend=showlegend,mode='lines'),
                         row=add_row,col=add_col)
vn_bp_fig.add_trace(go.Scatter(name="S",line={'color':'green'},legendgroup='10',mode='lines'),row=6,col=1)
vn_bp_fig.add_trace(go.Scatter(name="V",line={'color':'blue'},legendgroup='10',mode='lines'),row=6,col=1)
vn_bp_fig.add_trace(go.Scatter(name="I",line={'color':'orange'},legendgroup='10',mode='lines'),row=6,col=1)

##create buttons data for dropdown list

update_menu=[]
province_list = vn_bp_df[['id','provinceName']].drop_duplicates(ignore_index=True)
annotation_list = []
for note in vn_bp_fig.layout.annotations:
    annotation_list.append(go.layout.Annotation(note))
    
for code in vn_bp_predict_df.id.unique():
    if vn_bp_start_date is not None:
        vn_bp_start_stamp = pd.Timestamp(vn_bp_start_date)
        real_data = (vn_bp_df.loc[vn_bp_df.id==code]
                     .loc[(vn_bp_df.date<=vn_bp_start_stamp)&(vn_bp_df.date>=(vn_bp_start_stamp-pd.Timedelta(vn_bp_plot_point,unit="D")))]
                     .iloc[-vn_bp_plot_point:,vn_col_loc1].to_numpy())
    else:
        vn_bp_start_stamp = pd.Timestamp(vn_bp_df.loc[vn_bp_df.id==code].date.values[-1])
        real_data = vn_bp_df.loc[vn_bp_df.id==code].iloc[-vn_bp_plot_point:,vn_col_loc1].to_numpy()
    
    #forecast data
    if vn_bp_best_model:
        forecast_data = vn_bp_predict_df.loc[vn_bp_predict_df.id==code].iloc[:,vn_col_loc2].to_numpy()
    else: #if no best model is chosen, the data of the first model will be chosen
        forecast_data = vn_bp_predict_df.loc[(vn_bp_predict_df.id==code)&(vn_bp_predict_df.method==vn_bp_model_number[0])].iloc[:,vn_col_loc2].to_numpy()
    
    #get reproduction rate
    vn_bp_R0 = vn_bp_df.loc[vn_bp_df.id==code].loc[vn_bp_df.date==vn_bp_start_stamp,['daily_R0', 'avg_R0']].to_numpy().squeeze().tolist()
        
    x_list=[]
    x_list.extend([real_data[:,0]]*10)
    x_list.extend([forecast_data[:,0]]*10)
    x_list.extend([vn_bp_equil_df.loc[vn_bp_equil_df.id==code,'date'].to_numpy()]*3)
    
    y_list = []
    y_list.extend(real_data[:,1:].T)
    y_list.extend(forecast_data[:,1:].T)
    y_list.extend(vn_bp_equil_df.loc[vn_bp_equil_df.id==code,['S','V','I']].to_numpy().T)
    
    #The main title of each button
    plot_title=province_list.loc[province_list.id==code,'provinceName'].values[0]
    
    button_annotation = annotation_list.copy()
    button_annotation[10] = go.layout.Annotation(vn_bp_fig.layout.annotations[10]).update(text=f'Equilibrium plot (daily_R0:{round(vn_bp_R0[0],2)} - average_R0:{round(vn_bp_R0[1],2)})')
    button =  dict(label=plot_title,
                   method="update",
                   args=[{"y":y_list,
                          "x":x_list},
                         {"title":{"text":plot_title,"x":0.5,"font":{"size":50}},
                          "annotations":button_annotation}
                   ]
                  )
    update_menu.append(button)
    
##plotting

vn_bp_fig.update_layout(updatemenus=[dict(active=-1,
                                      buttons=update_menu,
                                      pad={"r": 10, "t": 10},
                                      showactive=True,
                                      xanchor="left",
                                      x=0.05,
                                      yanchor="top",
                                      y=1.05)
                                  ],
                     legend_tracegroupgap=285,
                     height=2000
                    )
clear_output()
vn_bp_fig.show()